In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.13


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [12]:
year = 2023
month = 3

In [6]:
df = read_data(f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet")

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
result_df = pd.DataFrame()

In [9]:
result_df["estimated_duration"] = y_pred

In [10]:
result_df.describe()

,estimated_duration
count,3.316216e+06
mean,1.420387e+01
std,6.247490e+00
min,-1.160751e+01
25%,1.096814e+01
50%,1.197354e+01
75%,1.413772e+01
max,7.498748e+01


In [13]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [15]:
result_df["ride_id"] = df['ride_id']

In [16]:
!mkdir outputs

In [17]:
output_file = f"outputs/yellow_tripdata_{year:04d}-{month:02d}.parquet"

In [19]:
result_df.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [20]:
result_df

,estimated_duration,ride_id
0,16.245906,2023/03_0
1,26.134796,2023/03_1
2,11.884264,2023/03_2
3,11.997720,2023/03_3
4,10.234486,2023/03_4
...,...,...
3316211,11.952635,2023/03_3316211
3316212,20.049958,2023/03_3316212
3316213,11.595336,2023/03_3316213
3316214,13.113178,2023/03_3316214
